# Particle Position Creation

- To process the `uncoupledKinematicParcelFoam` Lagrangian Solver within openFOAM, a text file containing positions to generate particles is needed within the VTK format. 

- This notebook generates these text files and creates a homogeneous spaced 'cloud' centered on the location of the Cloud Droplet Probe sample volume within the model

- Through testing, the time to simulate 50 thousand particles with `uncoupledKinematicParcelFoam` for the NASA P-3 with pylons (on a single processor) was four days on aircraft. Therefore, to create a homogenous 'cloud' across an entire grid cell for the solution would require 2 million particles (40 days per solution to solve). 

- Therefore, a much narrow portion of inlet patch cell is used to generate ~100#/cc concentrations with 223 x 223 particles (or 49729 total).

In [1]:
import numpy as np

## Define Inputs and Constants

In [2]:
# Note: particles are released at inlet patch, x-dimension not needed
sample_volume = {'navy' : [0, 2.74, 13.75],
                 'extended' : [0, 3.05, 13.7]
                }
# Define the number of particles
nparticle = 50000.
# Define the desired droplet concentration
droplet_conc = 100.

## Define the Mean Distance between Droplets for a given Concentration

- From this blog by Dr. Alex DeCaria of Millersville University on the [Mean Distance between Cloud Droplets](https://blogs.millersville.edu/adecaria/files/2021/11/Mean_Distance_Between_Cloud_Drops.pdf), the mean cloud droplet spacing for our ~100#/cc concentration is determined by $$\bar{r} = 0.554n^{-\frac{1}{3}}$$

- Small note: [this LaTeX generator was used to define the above equation](https://latexeditor.lagrida.com/)

In [3]:
# mean distance between droplets [convert to meters]
mean_dis = (0.554 * (droplet_conc**(-1/3.))) * 0.01 # distance in meters

In [4]:
mean_dis

0.0011935568182776639

## Calculate the Grid Area and center on CDP Sample Volume

In [5]:
# Define the area we are creating grid over. 
# For ~50k particles, we need ~500cc volume to generate 100/cc concentration
dy = int(np.round(np.sqrt(nparticle))) * mean_dis
print(dy)

0.2673567272941967


In [6]:
dy / 2

0.13367836364709834

In [7]:
navy_jy = np.arange(sample_volume['navy'][1] - (dy/2.), sample_volume['navy'][1] + (dy / 2.), mean_dis)
navy_jz = np.arange(sample_volume['navy'][2] - (dy/2.), sample_volume['navy'][2] + (dy / 2.), mean_dis)

In [8]:
navy_y, navy_z = np.meshgrid(navy_jy, navy_jz)

In [9]:
extend_jy = np.arange(sample_volume['extended'][1] - (dy/2.), sample_volume['extended'][1] + (dy / 2.), mean_dis)
extend_jz = np.arange(sample_volume['extended'][2] - (dy/2.), sample_volume['extended'][2] + (dy / 2.), mean_dis)

In [10]:
extend_y, extend_z = np.meshgrid(extend_jy, extend_jz)

### Write out the files

In [11]:
def write_header(nfile):
    """
    openFOAM requires specific file header information.
    This function takes care of that
    """
    # Write header information
    nfile.write("FoamFile\n")
    nfile.write("{\n")
    nfile.write("\tversion\t\t2.0;\n")
    nfile.write("\tformat\t\tascii;\n")
    nfile.write("\tclass\t\tvectorField;\n")
    nfile.write("\tlocation\tconstant;\n")
    nfile.write("\tobject\t\tpositions;\n")
    nfile.write("}\n")
    nfile.write("// * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * //\n")
    nfile.write("(")

In [12]:
# Create a file to output the data
navy_file = open(r"navyPylon_particle_positions_" + str(int(droplet_conc)) + 'cc.txt', "w")
extend_file = open(r"extendPylong_particle_positions_" + str(int(droplet_conc)) + 'cc.txt', "w")

In [13]:
# Navy File
write_header(navy_file)
for i in range(navy_y.shape[0]):
    for j in range(navy_z.shape[1]):
        navy_file.write('( -10.0 ' + str(np.around(navy_y[i, j], 5)) + ' ' + str(np.around(navy_z[i, j], 5)) + ')\n')
navy_file.write(')')
navy_file.close()

In [14]:
# Extended File
# Navy File
write_header(extend_file)
for i in range(extend_y.shape[0]):
    for j in range(extend_z.shape[1]):
        extend_file.write('( -10.0 ' + str(np.around(extend_y[i, j], 5)) + ' ' + str(np.around(extend_z[i, j], 5)) + ')\n')
extend_file.write(')')
extend_file.close()